In [4]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso
import warnings
warnings.filterwarnings('ignore')


def naive_nodewise_regression(Y_star, lambda_grid=None):
    """
    Implements Naive Nodewise Regression (Section 5.1.2).
    Uses GIC (Generalized Information Criterion) as in the paper.
    
    Parameters:
    -----------
    Y_star : np.ndarray, shape (n, p)
        Demeaned returns matrix (time x assets)
    lambda_grid : list or None
        Grid of lambda values to try. If None, creates default grid.
    
    Returns:
    --------
    Theta_hat : np.ndarray, shape (p, p)
        Estimated precision matrix
    """
    n, p = Y_star.shape

    # Initialize matrices
    Theta_hat = np.zeros((p, p))
    tau_squared = np.zeros(p)
    
    # Create lambda grid if not provided
    if lambda_grid is None:
        lambda_grid = np.logspace(-3, 1, 50)
    
    # For each asset j
    for j in range(p):
        # Step 1: Get y_j (target) and Y_{-j} (predictors)
        y_j = Y_star[:, j]
        Y_minus_j = np.delete(Y_star, j, axis=1)
        
        # Step 2-3: Estimate gamma_j using Lasso with GIC
        # GIC(λ) = log(σ²_λ) + |S_λ| * (log(p-1) / n) * log(log(n))
        best_gic = np.inf
        best_lambda = lambda_grid[0]
        best_gamma = None
        best_ssr = None
        
        for lam in lambda_grid:
            lasso = Lasso(alpha=2*lam, fit_intercept=False, max_iter=10000)
            lasso.fit(Y_minus_j, y_j)
            gamma_j = lasso.coef_
            
            # Compute SSR and number of non-zero coefficients
            residuals = y_j - Y_minus_j @ gamma_j
            ssr = np.sum(residuals ** 2)
            sigma_sq_lambda = ssr / n
            q_lambda = np.sum(gamma_j != 0)
            
            # Compute GIC
            if sigma_sq_lambda > 1e-10: # Check for non-zero variance
                # GIC formula from paper
                gic = np.log(sigma_sq_lambda) + q_lambda * (np.log(p - 1) / n) * np.log(np.log(n))
            else:
                gic = np.inf
            
            if gic < best_gic:
                best_gic = gic
                best_lambda = lam
                best_gamma = gamma_j.copy()
                best_ssr = ssr
        
        gamma_j_star = best_gamma
        

        tau_squared[j] = best_ssr / n + best_lambda * np.sum(np.abs(gamma_j_star))

        # [cite_start]Step 5: Form the j-th row of Theta_hat [cite: 579, 543-547]
        Theta_hat[j, j] = 1 / tau_squared[j]
        off_diag = -gamma_j_star / tau_squared[j]
        Theta_hat[j, :j] = off_diag[:j]
        Theta_hat[j, j+1:] = off_diag[j:]
    
    # Step 6: Symmetrize
    #Theta_hat_sym = (Theta_hat + Theta_hat.T) / 2
    
    return Theta_hat


def gmv_weights(Theta_hat, p):
    """
    Compute Global Minimum Variance (GMV) portfolio weights (Section 6.1).
    """
    ones_p = np.ones(p)
    
    # [cite_start]w* = (Θ 1_p) / (1_p' Θ 1_p) [cite: 818-819]
    numerator = Theta_hat @ ones_p
    denominator = ones_p @ Theta_hat @ ones_p
    
    w_star = numerator / denominator
    
    return w_star


def compute_portfolio_metrics(returns, weights):
    """
    Compute portfolio return, variance, and Sharpe ratio.
    """

    portfolio_returns = returns @ weights
    mean_return = np.mean(portfolio_returns)
    variance = np.var(portfolio_returns, ddof=1)
    sharpe_ratio = mean_return / np.sqrt(variance) if variance > 0 else 0
    
    return {
        'return': mean_return,
        'variance': variance,
        'sharpe_ratio': sharpe_ratio
    }


def backtest_nodewise_gmv(df, test_start_date='2000-01-31', test_end_date='2003-12-31',
                          lookback_window=180,
                          transaction_cost=0.001,
                          verbose=True): # <-- Added verbose parameter
    """
    Backtest Nodewise + GMV strategy with monthly rebalancing,
    180-month rolling window, and NaN filtering as per the paper.
    
    Parameters:
    -----------
    df : pd.DataFrame
        DataFrame with columns: permno, datadate, ret_fwd_1
    test_start_date : str
        First date for *out-of-sample* returns (format: 'YYYY-MM-DD')
    test_end_date : str
        Last date for *out-of-sample* returns (format: 'YYYY-MM-DD')
    lookback_window : int
        Number of months in rolling training window (default: 180)
    transaction_cost : float
        Proportional transaction cost (default: 0.005 = 50 bps)
    verbose : bool
        If True, prints detailed log at each time step.
    
    Returns:
    --------
    results_df : pd.DataFrame
        DataFrame with columns: date, portfolio_return, cumulative_return
    metrics : dict
        Overall performance metrics
    """
    # --- 1. Setup ---
    df = df.copy()
    if 'datadate' not in df.columns or 'permno' not in df.columns:
        raise ValueError("DataFrame must have 'datadate' and 'permno' columns")
    df['datadate'] = pd.to_datetime(df['datadate'])
    
    # Get unique dates and all assets
    all_dates = sorted(df['datadate'].unique())
    all_assets = sorted(df['permno'].unique())
    p_full = len(all_assets)
    
    # Convert test dates to datetime
    test_start_dt = pd.to_datetime(test_start_date)
    test_end_dt = pd.to_datetime(test_end_date)
    
    # Find date indices
    try:
        test_start_idx = all_dates.index(test_start_dt)
        test_end_idx = all_dates.index(test_end_dt)
    except ValueError as e:
        raise ValueError(f"Date not found in DataFrame: {e}")
    
    if test_start_idx < lookback_window:
        raise ValueError(f"Not enough data for lookback. Test start date {test_start_date} "
                         f"requires data back to {all_dates[test_start_idx - lookback_window]}, "
                         f"but only {test_start_idx} periods are available.")
    
    # Storage for results
    portfolio_returns = []
    portfolio_dates = []
    portfolio_weights = []
    portfolio_turnover = []
    
    # Initial weights (full universe, equal-weighted)
    prev_weights = np.ones(p_full) / p_full
    last_oos_returns = np.zeros(p_full)
    last_gross_return = 0.0
    
    # --- 2. Rolling Window Backtest ---
    if verbose:
        print("--- Starting Backtest ---")
        
    for t in range(test_start_idx, test_end_idx + 1):
        current_date = all_dates[t]
        
        # Define the 180-month (or lookback_window) in-sample period
        window_start_date = all_dates[t - lookback_window]
        window_end_date = all_dates[t - 1]
        
        # Get training data for this window
        train_data = df[(df['datadate'] >= window_start_date) & 
                        (df['datadate'] <= window_end_date)]
        
        # Pivot to get returns matrix (time x assets)
        returns_pivot = train_data.pivot(index='datadate', columns='permno', values='ret_fwd_1')
        
        # Reindex to ensure all dates are present (fills missing dates with NaNs)
        window_dates = all_dates[t - lookback_window : t]
        returns_pivot = returns_pivot.reindex(index=window_dates, columns=all_assets)
        
        # **Filter assets with any NaNs in this window**
        nan_assets = returns_pivot.columns[returns_pivot.isna().any()]
        filtered_pivot = returns_pivot.drop(columns=nan_assets)
        
        current_assets = filtered_pivot.columns.tolist()
        Y = filtered_pivot.values
        n_train, p_current = Y.shape
        
        new_weights = np.zeros(p_full) # Start with 0 weights for all assets

        if verbose:
            print("\n" + "="*50)
            print(f"Processing Date: {current_date.strftime('%Y-%m-%d')}")
            print(f"  In-Sample Window: {window_start_date.strftime('%Y-%m-%d')} to {window_end_date.strftime('%Y-%m-%d')}")
            print(f"  Asset Universe: {p_full} total, {p_current} with complete data in window")

        # Check for valid data panel
        if n_train < lookback_window or p_current < 2:
            if verbose:
                print(f"  Skipping estimation: insufficient data (n={n_train}, p={p_current}). Carrying forward weights.")
            new_weights = prev_weights 
        else:
            try:
                # Demean the returns
                Y_bar = Y.mean(axis=0)
                Y_star = Y - Y_bar
                
                if verbose: print(f"  Running Naive Nodewise for {p_current} assets...")
                Theta_hat = naive_nodewise_regression(Y_star)
                
                if verbose: print("  Calculating GMV weights...")
                w_star_filtered = gmv_weights(Theta_hat, p_current)
                
                # Map weights back to full asset universe
                new_weights_series = pd.Series(0.0, index=all_assets)
                new_weights_series.loc[current_assets] = w_star_filtered
                new_weights = new_weights_series.values
                
            except Exception as e:
                if verbose:
                    print(f"  Error at {current_date}: {e}. Carrying forward weights.")
                new_weights = prev_weights

        # Normalize weights to sum to 1
        weight_sum = np.sum(new_weights)
        if np.abs(weight_sum) > 1e-6:
            new_weights = new_weights / weight_sum
        else:
            if verbose: print("  Warning: Zero weight sum, carrying forward weights.")
            new_weights = prev_weights
        
        # --- 3. OOS Calculation & Transaction Costs ---
        
        # Get out-of-sample returns for current month (full universe)
        oos_data = df[df['datadate'] == current_date]
        oos_returns_pivot = oos_data.pivot(index='datadate', columns='permno', values='ret_fwd_1')
        oos_returns_pivot = oos_returns_pivot.reindex(columns=all_assets)
        oos_returns = oos_returns_pivot.values[0]
        valid = ~np.isnan(oos_returns)
        oos_returns = np.nan_to_num(oos_returns, nan=0.0)
        
        # Set weights of missing-return stocks to 0 and re-normalize
        new_weights = new_weights * valid
        if new_weights.sum() != 0:
            new_weights /= new_weights.sum()
        
        # Compute gross portfolio return (y_P,t+1)
        gross_return = new_weights @ oos_returns
        
        # Adjust previous weights (w_t,j+) as per paper
        adj_prev_weights = prev_weights * (1 + last_oos_returns) / (1 + last_gross_return)
        
        # Compute turnover and transaction costs
        turnover = np.sum(np.abs(new_weights - adj_prev_weights))
        tc = transaction_cost * (1 + gross_return) * turnover
        
        # Net return (after transaction costs)
        net_return = gross_return - tc
        
        # Store results
        portfolio_returns.append(net_return)
        portfolio_dates.append(current_date)
        w_temp = pd.Series(new_weights, index=all_assets)
        w_temp = w_temp[w_temp != 0]
        portfolio_weights.append(w_temp)

        portfolio_turnover.append(turnover)
        
        # Update "previous" variables for next loop
        prev_weights = new_weights.copy()
        last_oos_returns = oos_returns.copy()
        last_gross_return = gross_return
        
        if verbose:
            print(f"  Gross Return: {gross_return: .6f}")
            print(f"  Turnover:     {turnover: .6f}")
            print(f"  Trans. Cost:  {tc: .6f}")
            print(f"  Net Return:   {net_return: .6f}")
            
            print(portfolio_weights[-1])

    if verbose:
        print("="*50)
        print("--- Backtest Complete ---")
    
    # --- 4. Final Metrics ---
    results_df = pd.DataFrame({
        'date': portfolio_dates,
        'portfolio_return': portfolio_returns,
        'portfolio_weights': portfolio_weights,
        'portfolio_turnover': portfolio_turnover
    })
    results_df['cumulative_return'] = (1 + results_df['portfolio_return']).cumprod() - 1
    
    # Compute overall metrics
    mean_return = np.mean(portfolio_returns)
    variance = np.var(portfolio_returns, ddof=1)
    sharpe_ratio = mean_return / np.sqrt(variance) if variance > 0 else 0
    
    # Annualized metrics (monthly data)
    annual_return = mean_return * 12
    annual_volatility = np.sqrt(variance * 12)
    annual_sharpe = annual_return / annual_volatility if annual_volatility > 0 else 0
    
    metrics = {
        'mean_return': mean_return,
        'variance': variance,
        'sharpe_ratio': sharpe_ratio,
        'annual_return': annual_return,
        'annual_volatility': annual_volatility,
        'annual_sharpe_ratio': annual_sharpe,
        'total_return': results_df['cumulative_return'].iloc[-1] if len(results_df) > 0 else 0
    }
    
    return results_df, metrics

In [2]:
# Assuming you have loaded your dataframe as 'df'
df = pd.read_csv('../green cleaned.csv', dtype={'ncusip': 'string'})
df['ret_fwd_1'] = (df.groupby('permno')['ret_excess'].shift(-1) )

In [5]:
results_df, metrics = backtest_nodewise_gmv(
    df,
    test_start_date='2020-01-31',  # Last date of training period
    test_end_date='2024-11-30',   # Last date of testing period
    lookback_window=180,
    transaction_cost=0.001
)
print(f"Sharpe Ratio: {metrics['sharpe_ratio']:.4f}")
print(f"Annualized Sharpe Ratio: {metrics['annual_sharpe_ratio']:.4f}")

--- Starting Backtest ---

Processing Date: 2020-01-31
  In-Sample Window: 2005-01-31 to 2019-12-31
  Asset Universe: 1521 total, 252 with complete data in window
  Running Naive Nodewise for 252 assets...
  Calculating GMV weights...
  Gross Return: -0.093563
  Turnover:      1.674761
  Trans. Cost:   0.001518
  Net Return:   -0.095081
10104    0.004351
10107    0.003989
10138    0.003652
10145    0.006107
10516    0.002637
           ...   
89179    0.002983
89195    0.000181
89258    0.002025
89525    0.003865
92655    0.003338
Length: 252, dtype: float64

Processing Date: 2020-02-29
  In-Sample Window: 2005-02-28 to 2020-01-31
  Asset Universe: 1521 total, 252 with complete data in window
  Running Naive Nodewise for 252 assets...
  Calculating GMV weights...
  Gross Return: -0.141755
  Turnover:      0.034977
  Trans. Cost:   0.000030
  Net Return:   -0.141785
10104    0.004437
10107    0.004041
10138    0.003650
10145    0.006448
10516    0.002654
           ...   
89179    0.003

In [ ]:

with pd.option_context("display.max_rows", None):
    print(results_df['portfolio_weights'][2])

In [6]:
metrics['total_return']

0.42053866429671394

In [7]:
metrics['variance']

0.0025077048400304184

In [10]:
results_df['portfolio_turnover'].mean()

0.08298816230359735